# B. State purification with qubit-mediated measurement

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [2]:
from feedback_grape.fgrape import optimize_pulse
import jax.numpy as jnp

## The cavity is initially in a  mixed state --> Goal is to purify the state

We are trying to maximize the property determined by $tr (\rho_{\text{cav}}^2)$ which is the purity

In the following, we consider an adaptive measurement
scheme, demonstrated in a series of experiments on Rydberg atoms interacting
with microwave cavities. In this scheme, the
cavity is coupled to an ancilla qubit, which can then be
read out to update our knowledge of the quantum state of
the cavity. <br> <br>
<p align="center">
    <img src="../../_static/notebook-images/b_tut.png" alt="image" width="350"/>
</p>


In [3]:
# initial state is a thermal state
n_average = 2
N_cavity = 30
# natural logarithm
beta = jnp.log((1 / n_average) + 1)
diags = jnp.exp(-beta * jnp.arange(N_cavity))
normalized_diags = diags / jnp.sum(diags, axis=0)
rho_cav = jnp.diag(normalized_diags)

### Now the thing is here, we don't need a rho_final because the purity or the reward that we want to maximize is $tr (\rho_{\text{cav}}^2)$.
Unlike fidelity expressions which wants to find how close to states are

## Next Step is to construct our POVM

In [4]:
from feedback_grape.utils.operators import cosm, sinm

### povm_measure_operator (callable): <br>
    - It should take a measurement outcome and list of params as input
    - The measurement outcome options are either 1 or -1

In [5]:
from feedback_grape.utils.operators import create, destroy, identity
import jax


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma = params[0]
    delta = params[1]
    number_operator = create(N_cavity) @ destroy(N_cavity)
    angle = (gamma * number_operator) + delta / 2 * identity(N_cavity)
    return jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )

In [6]:
# Answer: why does RNN outputs two the same, then two the same, then two the same and so on? -> calculate during forward proagation then again during back_propagation
from feedback_grape.fgrape import Gate

measure = Gate(
    gate=povm_measure_operator,
    initial_params=jax.random.uniform(
        key=jax.random.PRNGKey(42),
        shape=(2,),
        minval=0.0,
        maxval=jnp.pi,
        dtype=jnp.float64,
    ),
    measurement_flag=True,
    # param_constraints=[
    #     [0, jnp.pi],
    #     [-2 * jnp.pi, 2 * jnp.pi],
    # ],
)

system_params = [measure]

result = optimize_pulse(
    U_0=rho_cav,
    C_target=None,
    system_params=system_params,
    num_time_steps=5,
    mode="lookup",
    goal="purity",
    max_iter=1000,
    convergence_threshold=1e-20,
    learning_rate=0.01,
    evo_type="density",
    batch_size=10,
)

In [7]:
print(result.final_fidelity)

None


In [8]:
# 0.9163363647226792
print(result.final_purity)

0.9725933867416181


In [9]:
from feedback_grape.utils.purity import purity

# the highest purity can be 0.995 if the initial params that initializes the lookup table
# are between 0 and pi rather than -pi and pi
print("initial purity:", purity(rho=rho_cav))
for i, state in enumerate(result.final_state):
    print(f"Purity of state {i}:", purity(rho=state))

initial purity: 0.20000208604889932
Purity of state 0: 0.9701125120075851
Purity of state 1: 0.9852663136664356
Purity of state 2: 0.9612230397524459
Purity of state 3: 0.9975439279038868
Purity of state 4: 0.9612230397524459
Purity of state 5: 0.9386428333927941
Purity of state 6: 0.9852663136664356
Purity of state 7: 0.9612230397524459
Purity of state 8: 0.9711236930463625
Purity of state 9: 0.9943091544753442


In [10]:
result.returned_params

[[Array([[1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029],
         [1.09333296, 0.05908029]], dtype=float64)],
 [Array([[1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 1.48349559e-16],
         [1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 1.48349559e-16],
         [1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 1.48349559e-16],
         [1.57079633e+00, 1.48349559e-16],
         [1.09323741e+00, 4.51206584e-01],
         [1.09323741e+00, 4.51206584e-01],
         [1.57079633e+00, 1.48349559e-16]], dtype=float64)],
 [Array([[ 1.57079633e+00,  3.10262901e-15],
         [ 1.02955772e+00, -8.74809684e-03],
         [ 1.57079633e+00,  3.10262901e-15],
         [ 1.40156193e+00,  7.14115220e-01],
    